# The Greenhouse Gas (GHS) emissions

In [21]:
# HIDDEN
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get GHG emission from agricultural land-use

In [22]:
# HIDDEN
import altair as alt

from tools import get_all_files, get_GHG_file_df
from tools.Plot_GHG import get_GHG_plots

## Data preparing

In [44]:
# HIDE CODE
# Get all output files
from PARAMETERS import DATA_ROOT
files = get_all_files(DATA_ROOT)


# Select files based on criteria
files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique(),'\n')

['GHG' 'dvar' 'ammap' 'lmmap' 'lumap_separate_Agricultural Land-use'
 'lumap_separate_Non-Agricultural Land-use' 'lumap' 'water' 'cross_table'
 'quantity'] 



Filter the GHG data

In [24]:
# NO CODE
# Get the GHG files
GHG_files = get_GHG_file_df(files_selc)
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files.head(3)

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
2,2010,GHG,Non-Agricultural Landuse,.csv,/home/docker_data/LUTO_DATA/2023_11_21__05_26_...,-107778.2,-0.107778
1,2010,GHG,Agricultural Management,.csv,/home/docker_data/LUTO_DATA/2023_11_21__05_26_...,0.0,0.000000
3,2010,GHG,Transition Penalty,.csv,/home/docker_data/LUTO_DATA/2023_11_21__05_26_...,0.0,0.000000


Calculate the net emissions

In [49]:
# HIDDEN
# Calculate the net emissions
Net_emission = GHG_files.groupby('year')['GHG_sum_Mt'].sum(numeric_only = True).reset_index()
Net_emission = Net_emission.rename(columns={'GHG_sum_Mt':'Net_emission'})

Net_emission['Net_emission_cum'] = Net_emission['Net_emission'].cumsum()
Net_emission.head(3)

,year,Net_emission,Net_emission_cum
0,2010,90.404125,90.404125
1,2011,79.323406,169.727530
2,2012,68.648003,238.375533


## Total GHG emissions across years

In [121]:
# HIDE CODE

# Create a base chart with the necessary transformations and encodings
column_chart_individual = alt.Chart(
    GHG_files
).mark_bar().encode(
    tooltip=[alt.Tooltip('base_name', title='GHG Category'),
             alt.Tooltip('GHG_sum_Mt:Q', title='Emissions (Mt CO2e)')],
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)), 
    y=alt.Y('GHG_sum_Mt:Q',title='Emissions (Mt CO2e)'),  
    color=alt.Color('base_name:N',legend=alt.Legend(
                                            title=None,
                                            orient='none',
                                            legendX=810, legendY=200,
                                            direction='vertical',
                                            titleAnchor='middle')), 
    
)



# Create a column chart to show the cumsum emissions
column_chart_cumsum = alt.Chart(
    Net_emission
).mark_bar(
    color='grey',opacity=0.2
).transform_calculate(
    Cumulative_emissions =  'datum.year > 0 ? "Cumulative emissions" : "Cumulative emissions"'
).encode(  
    tooltip=[alt.Tooltip('Net_emission_cum:Q',title='Cumulative Quantity (Mt CO2e)')],
    color=alt.Color('Cumulative_emissions:N',
                    scale=alt.Scale(range=['#181818'])  ,
                    legend=alt.Legend(title=None,
                                      orient='none',
                                      legendX=810, legendY=270,
                                      direction='vertical',
                                      titleAnchor='middle')),
    x=alt.X('year:O'),
    y=alt.Y('Net_emission_cum:Q',title='Emissions (Mt CO2e)'), 
)




# Create a line chart for net emissions
line_chart_net = alt.Chart(
    Net_emission
).transform_calculate(
    Net_emission_color =  'datum.year > 0 ? "Net emissions" : "Net emissions"'
).mark_line(
    opacity=0.7
).encode(
    tooltip=[alt.Tooltip('Net_emission:Q',title='Net Quantity (Mt CO2e)')],
    color=alt.Color('Net_emission_color:N',legend=alt.Legend(
                                            title=None,
                                            orient='none',
                                            legendX=810, legendY=290,
                                            direction='vertical',
                                            titleAnchor='middle')),
    x=alt.X('year:O'),
    y=alt.Y('Net_emission:Q',title='Emissions (Mt CO2e)'))




# Combine the layers into a final chart
final_chart = alt.layer(
    column_chart_cumsum,
    column_chart_individual,
    line_chart_net,
).properties(
    width=800,
    height=450,
).resolve_scale(
    color='independent'
)


final_chart


alt.LayerChart(...)

## Agricultural Land use Emissions

Get the Agricultural land-use data

In [27]:
# HIDDEN
# Initialize the class
ag_lucc_GHG = get_GHG_plots(GHG_files,'Agricultural Landuse','GHG emissions (Mt CO2e)')

### Emissions by Crops/Livestocks

In [28]:
# HIDE CODE
# get the df and plot
GHG_crop_lvstk_df,GHG_crop_lvstk_plot = ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_crop_lvstk_plot

alt.LayerChart(...)

### Emissions by differrnt irrigation system

In [29]:
# HIDE CODE
# get the df and plot
GHG_dry_irr_df,GHG_dry_irr_plot = ag_lucc_GHG.plot_GHG_dry_irr()
GHG_dry_irr_plot

alt.LayerChart(...)

### Emissions by GHG category

In [30]:
# HIDE CODE
# get the df and plot
GHG_catetory_df,GHG_atetory_plot = ag_lucc_GHG.plot_GHG_category()
GHG_atetory_plot

alt.LayerChart(...)

### Emission by CO2 sources

In [31]:
# HIDE CODE
# get the df and plot
GHG_sources_df,GHG_sources_plot = ag_lucc_GHG.plot_GHG_sources()
GHG_sources_plot

alt.LayerChart(...)

### Emissions by {land-use}-{irrigation} cmobined

In [32]:
# HIDE CODE
# get the df and plot
GHG_lu_lm_df,GHG_lu_lm_plot = ag_lucc_GHG.plot_GHG_lu_lm(2050)
GHG_lu_lm_plot

alt.Chart(...)

### Emissions by {land-use}-{GHG source} combined

In [33]:
# HIDE CODE
# get the df and plot
GHG_lu_source_df,GHG_lu_source_plot = ag_lucc_GHG.plot_GHG_lu_source(2050)
GHG_lu_source_plot

alt.Chart(...)

## Non-Agricultural Land-use GHG sequestrations

### Data preparation

In [34]:
# HIDDEN
# Initialize the class
non_ag_lucc_GHG = get_GHG_plots(GHG_files,'Non-Agricultural Landuse','GHG sequestration (Mt CO2e)')

/home/LUTO/VIS_LUTO/tools/Plot_GHG.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([GHG_CO2,GHG_nonCO2],axis=0).reset_index(drop=True)


### Sequestrations in total

In [35]:
# HIDE CODE
# get the df and plot
GHG_non_ag_crop_lvstk_df,GHG_non_ag_crop_lvstk_plot = non_ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_non_ag_crop_lvstk_plot

alt.LayerChart(...)

### Sequestrations by irrigation system

In [36]:
# HIDE CODE
# get the df and plot
GHG_non_ag_dry_irr_df,GHG_non_ag_dry_irr_plot = non_ag_lucc_GHG.plot_GHG_dry_irr()
GHG_non_ag_dry_irr_plot

alt.LayerChart(...)

## Agricultural Management GHG sequestrations

### Data preparation

In [37]:
# HIDDEN
# Initialize the class
ag_man_GHG = get_GHG_plots(GHG_files,'Agricultural Management','GHG sequestration (Mt CO2e)')

/home/LUTO/VIS_LUTO/tools/Plot_GHG.py:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([GHG_CO2,GHG_nonCO2],axis=0).reset_index(drop=True)


In [38]:
ag_man_GHG.GHG_df_long

,Year,Land use category,Land use,Irrigation,Sources,Quantity (Mt CO2e),GHG Category
0,2010,Crop,Apples,dry,TCO2E_Asparagopsis taxiformis,0.0,CO2
1,2010,Crop,Citrus,dry,TCO2E_Asparagopsis taxiformis,0.0,CO2
2,2010,Crop,Cotton,dry,TCO2E_Asparagopsis taxiformis,0.0,CO2
3,2010,Crop,Grapes,dry,TCO2E_Asparagopsis taxiformis,0.0,CO2
4,2010,Crop,Hay,dry,TCO2E_Asparagopsis taxiformis,0.0,CO2
...,...,...,...,...,...,...,...
5623,2050,Crop,Winter legumes,irr,TCO2E_Ecological Grazing,0.0,CO2
5624,2050,Crop,Winter oilseeds,irr,TCO2E_Ecological Grazing,0.0,CO2
5625,2050,Livestock,Beef,irr,TCO2E_Ecological Grazing,0.0,CO2
5626,2050,Livestock,Dairy,irr,TCO2E_Ecological Grazing,0.0,CO2


### Sequestrations by Crops/Livestocks

In [39]:
# HIDE CODE
# get the df and plot
GHG_ag_man_GHG_crop_lvstk_df,GHG_ag_man_GHG_crop_lvstk_plot = ag_man_GHG.plot_GHG_crop_lvstk()
GHG_ag_man_GHG_crop_lvstk_plot

alt.LayerChart(...)

### Sequestrations by irrigation system

In [40]:
# HIDE CODE
# get the df and plot
GHG_ag_man_dry_irr_df,GHG_ag_man_dry_irr_plot = ag_man_GHG.plot_GHG_dry_irr()
GHG_ag_man_dry_irr_plot

alt.LayerChart(...)

### Sequestrations by GHG category

In [41]:
# HIDE CODE
# get the df and plot
GHG_ag_man_df,GHG_ag_man_plot = ag_man_GHG.plot_GHG_category()
GHG_ag_man_plot

alt.LayerChart(...)